In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

YEARS_FAR_BACK = 25
END_DATE = datetime.now().strftime("%Y-%m-%d")
START_DATE = (datetime.now() - timedelta(days=YEARS_FAR_BACK*365)).strftime("%Y-%m-%d")
PAGE_SIZE = 1000

def get_fiscal_data(endpoint, params):
    base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service"
    url = f"{base_url}/{endpoint}"
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()['data']
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

In [2]:
def get_treasury_data():
    # Initialize an empty list to hold data
    all_debt_data = []
    all_exp_rev_data = []

    # Iterate through pages 1 to 100
    for page_number in range(1, 8):
        # Fetch debt data
        debt_data = get_fiscal_data(
            "v2/accounting/od/debt_to_penny",
            {
                "fields": "record_date,tot_pub_debt_out_amt",
                "filter": f"record_date:gte:{START_DATE},record_date:lte:{END_DATE}",
                "sort": "-record_date",
                "page[number]": page_number,
                "page[size]": PAGE_SIZE,
            }
        )
        all_debt_data.extend(debt_data)
        # Fetch expenditure and revenue data

    for desc in ["Total Outlays", "Total Receipts", "Surplus (+) or Deficit (-)"]:
        monthly_finances = get_fiscal_data(
                    "/v1/accounting/mts/mts_table_3",
                    {
                        "fields": "record_date,current_month_rcpt_outly_amt, classification_desc",#, current_month_gross_outly_amt, current_month_dfct_sur_amt",
                        "filter": f"record_date:gte:{START_DATE},record_date:lte:{END_DATE},classification_desc:eq:{desc}",
                        "sort": "-record_date",
                        "page[number]": 1,
                        "page[size]": 1000,
                    }
                )
        all_exp_rev_data.extend(monthly_finances)
    debt_data_df = pd.DataFrame(all_debt_data)
    monthly_df = pd.DataFrame(all_exp_rev_data)
    return debt_data_df, monthly_df


def process_treasury_data(debt_data_df, monthly_df):
    # Convert the record_date column to a datetime object
    debt_data_df['record_date'] = pd.to_datetime(debt_data_df['record_date'])
    monthly_df['record_date'] = pd.to_datetime(monthly_df['record_date'])

    pivoted_df = monthly_df.drop_duplicates().pivot(index='record_date',
                                                    columns='classification_desc', values='current_month_rcpt_outly_amt')
    pivoted_df = pivoted_df.reset_index()
    pivoted_df['month-year'] = pd.to_datetime(pivoted_df['record_date']).dt.to_period('M')
    debt_data_df['month-year'] = pd.to_datetime(debt_data_df['record_date']).dt.to_period('M')
    debt_data_df = debt_data_df.groupby('month-year').first().reset_index()
    debt_data_df = debt_data_df.drop(columns=['record_date'])
    pivoted_df = pivoted_df.drop(columns=['record_date'])

    # Merge the two dataframes
    merged_df = pd.merge(debt_data_df, pivoted_df, on='month-year', how='right')
    # Convert object columns to numeric
    merged_df['tot_pub_debt_out_amt'] = pd.to_numeric(merged_df['tot_pub_debt_out_amt'], errors='coerce')
    merged_df['Surplus (+) or Deficit (-)'] = pd.to_numeric(merged_df['Surplus (+) or Deficit (-)'], errors='coerce')
    merged_df['Total Outlays'] = pd.to_numeric(merged_df['Total Outlays'], errors='coerce')
    merged_df['Total Receipts'] = pd.to_numeric(merged_df['Total Receipts'], errors='coerce')
    return merged_df


In [3]:
debt_data_df, monthly_df = get_treasury_data()
merged_df = process_treasury_data(debt_data_df, monthly_df)
merged_df.to_csv("../data/treasury_data.csv", index=False)
merged_df.head()

,month-year,tot_pub_debt_out_amt,Surplus (+) or Deficit (-),Total Outlays,Total Receipts
0,2015-03,1.815206e+13,-5.291006e+10,2.870974e+11,2.341873e+11
1,2015-04,1.815256e+13,1.567141e+11,3.150872e+11,4.718012e+11
2,2015-05,1.815285e+13,-8.238528e+10,2.947711e+11,2.123858e+11
3,2015-06,1.815200e+13,5.177576e+10,2.911573e+11,3.429331e+11
4,2015-07,1.815132e+13,-1.491868e+11,3.746801e+11,2.254934e+11
